In [29]:
include("../../src/WaterLily.jl")

using Plots
using Statistics
using JLD

In [30]:
optPois = "MulP"
optSour = "Cylinder"
optBCon = "Per"

stepbystep = false

computationID = optPois * "_" * optSour * "Low" * "_" * optBCon * "_"
print(computationID,"\n")

MulP_CylinderLow_Per_


In [31]:
T=Float32
N = (5*2^8,2^8)
#N = (2^8,2^8)
D = length(N)
Ng = N .+ 2
Nd = (Ng..., 2)
figureSize = Int.(N.*3.75)

l = N[2]
xc = N[1]/2+1.5
yc = 0+1.5
R = 2^6
U=1

optSour=="Cylinder" && function SourceFunc(x,y)
    r = sqrt.((x.-xc).^2+(y.-yc).^2)

    return -32*pi^6*R^4*U^2/l^6*(
        (cosh.(2*pi*(x.-xc)/l).+cos.(2*pi*(y.-yc)/l))./
        (cosh.(2*pi*(x.-xc)/l).-cos.(2*pi*(y.-yc)/l)).^3
    ).*(r .> R)
end

optSour=="Cylinder" && function ManufacSol(x,y)
    r = sqrt.((x.-xc).^2+(y.-yc).^2)

    return 2*R^2*pi^2*U^2/l^4*(
        (l^2*cosh.(2*pi*(x.-xc)/l).*cos.(2*pi*(y.-yc)/l).-pi^2*R^2 .-l^2)./
        (cosh.(2*pi*(x.-xc)/l).-cos.(2*pi*(y.-yc)/l)).^2
    ).*(r .> R)
end;

optSour=="Cylinder" && function UGradU(i, xyz)
    x,y = @. (xyz.- [xc,yc])*2*pi/l
    coeff = 8*U^2*R^2*pi^3/l^5 ./ (-cosh.(x).+cos.(y)).^3
    commonInner = pi^2*R^2+l^2 .- l^2*cos(y).*cosh(x)/2
    i==1 && return coeff .*(commonInner .- l^2* cos(y).^2/2).*sinh(x) # u_x
    i==2 && return coeff .*(commonInner .- l^2*cosh(x).^2/2).* sin(y) # u_y
    return 0.                              # u_z
end;

optSour=="OneCylinder" && function SourceFunc(x,y)
    r = sqrt.((x.-xc).^2+(y.-yc).^2)

    return -8*U^2*R^4 ./r.^6 .*(r .> 0.99R)
end

optSour=="OneCylinder" && function ManufacSol(x,y)
    r = sqrt.((x.-xc).^2+(y.-yc).^2)

    return -U^2*R^2/2*(R^2 .-2*(x.-xc).^2 .+2*(y.-yc).^2)./r.^4 .*(r .> 0.99R)
end;

In [32]:
#optPois=="MulP" && WaterLily.BC!(μ₀,ntuple(zero, D))

X = transpose(reshape([i-0.5 for i=1:Ng[1] for j=1:Ng[2]],Ng[2],Ng[1]))
Y = transpose(reshape([j-0.5 for i=1:Ng[1] for j=1:Ng[2]],Ng[2],Ng[1]))

source = SourceFunc(X,Y);
realSol = ManufacSol(X,Y);
originalSource = copy(source);
solIni = rand(Ng...);

WaterLily.BCPer!(solIni)
# WaterLily.BCPer!(source)
# WaterLily.BC(source)
# BCPerVec!(μ₀)

In [33]:
body = WaterLily.AutoBody((x,t)->√sum(abs2, x .- Array([xc,yc])) - R);
body+= WaterLily.AutoBody((x,t)->√sum(abs2, x .- Array([xc,yc+l])) - R);
flow = WaterLily.Flow(N,(0,0);Δt=1.0);
flow.μ₀ .= 1.0;
WaterLily.measure!(flow,body;ϵ=1);
μ₀ = flow.μ₀
#WaterLily.BC!(μ₀,(1,1))

1282×258×2 Array{Float64, 3}:
[:, :, 1] =
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 ⋮                        ⋮              ⋱                      ⋮         
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0 

In [34]:
WaterLily.apply!(UGradU, flow.f)
WaterLily.BDIM!(flow)
WaterLily.BCPerVec!(flow.u)


In [35]:

BDIMSource = zeros(size(source))
for i∈2:N[1]+1, j∈2:N[2]+1
    I = CartesianIndex(i,j)
    BDIMSource[I] = -WaterLily.div(I,flow.u)
end
WaterLily.BCPer!(BDIMSource)
OriginalBDIMSource = copy(BDIMSource)

1282×258 Matrix{Float64}:
 1.82514e-8   1.82514e-8   1.82404e-8   …  1.82514e-8   1.82514e-8
 1.12117e-14  1.12117e-14  1.12048e-14     1.12117e-14  1.12117e-14
 1.14859e-14  1.14859e-14  1.14789e-14     1.14859e-14  1.14859e-14
 1.17668e-14  1.17668e-14  1.17596e-14     1.17668e-14  1.17668e-14
 1.20543e-14  1.20543e-14  1.2047e-14      1.20543e-14  1.20543e-14
 1.23488e-14  1.23488e-14  1.23413e-14  …  1.23488e-14  1.23488e-14
 1.26504e-14  1.26504e-14  1.26426e-14     1.26504e-14  1.26504e-14
 1.29591e-14  1.29591e-14  1.29512e-14     1.29591e-14  1.29591e-14
 1.32753e-14  1.32753e-14  1.32672e-14     1.32753e-14  1.32753e-14
 1.3599e-14   1.3599e-14   1.35907e-14     1.3599e-14   1.3599e-14
 ⋮                                      ⋱               
 1.32753e-14  1.32753e-14  1.32672e-14     1.32753e-14  1.32753e-14
 1.29591e-14  1.29591e-14  1.29512e-14     1.29591e-14  1.29591e-14
 1.26504e-14  1.26504e-14  1.26426e-14  …  1.26504e-14  1.26504e-14
 1.23488e-14  1.23488e-14  1.23413e

In [36]:
optPois=="Pois" && (pTest = WaterLily.Poisson(solIni, μ₀, BDIMSource));
optPois=="MulP" && (pTest = WaterLily.MultiLevelPoisson(solIni, μ₀, BDIMSource));

In [37]:
stepbystep && for i ∈ 1:200
    partRes = WaterLily.solver!(pTest;log=true,tol=1e-12,itmx=20)
    # pTest.x .-= mean(pTest.x[2:end-1,2:end-1]) 
    pythonplot()
    contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],pTest.x[2:end-1,2:end-1];levels=32)
    savefig("bunchFig/" * computationID * "Calculated_"*string(i, pad=4)*".png")
    # pythonplot()
    # contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(realSol[2:end-1,2:end-1]-pTest.x[2:end-1,2:end-1]).+1e-12);levels=32)
    # savefig("bunchFig/" * computationID * "Error_"*string(i, pad=4)*".png")

    push!(res,partRes...)
end

!stepbystep && (res = WaterLily.solver!(pTest;log=true,tol=1e-12,itmx=1e3));

In [38]:
#print(res)
r = min.(sqrt.((X.-xc).^2+(Y.-yc).^2),sqrt.((X.-xc).^2+(Y.-yc.-l).^2));
pTest.x .-= mean(pTest.x[2,2:end-1]);
realSol .-= mean(realSol[2,2:end-1]);
pTest.x .*= (r .> R);
realSol .*= (r .> R);
mu = (μ₀[2:end-1,2:end-1,1]+μ₀[3:end,2:end-1,1]+μ₀[2:end-1,2:end-1,2]+μ₀[2:end-1,3:end,2])/4
errors = realSol[2:end-1,2:end-1]-pTest.x[2:end-1,2:end-1]

optPois=="Pois" && (lc = pTest.r)
optPois=="MulP" && (lc = pTest.levels[1].r)

1282×258 Matrix{Float64}:
 0.0   0.0           0.0          …   0.0           0.0          0.0
 0.0  -1.64257e-11  -1.64257e-11     -1.64257e-11  -1.64257e-11  0.0
 0.0  -1.35774e-11  -1.35774e-11     -1.35774e-11  -1.35774e-11  0.0
 0.0  -1.36773e-11  -1.36773e-11     -1.36773e-11  -1.36773e-11  0.0
 0.0  -9.99196e-12  -9.99196e-12     -9.99196e-12  -9.99196e-12  0.0
 0.0  -1.48923e-11  -1.48923e-11  …  -1.48923e-11  -1.48923e-11  0.0
 0.0  -1.11463e-11  -1.11463e-11     -1.11463e-11  -1.11463e-11  0.0
 0.0  -1.12545e-11  -1.12545e-11     -1.12545e-11  -1.12545e-11  0.0
 0.0  -8.11825e-12  -8.11825e-12     -8.11825e-12  -8.11825e-12  0.0
 0.0  -1.56974e-11  -1.56974e-11     -1.56974e-11  -1.56974e-11  0.0
 ⋮                                ⋱   ⋮                          
 0.0  -1.63189e-11  -1.63189e-11     -1.63189e-11  -1.63189e-11  0.0
 0.0  -1.33961e-11  -1.33961e-11     -1.33961e-11  -1.33961e-11  0.0
 0.0  -1.34988e-11  -1.34988e-11  …  -1.34988e-11  -1.34988e-11  0.0
 0.0  -9.96

In [39]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],originalSource[2:end-1,2:end-1];levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "Source.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/MulP_CylinderLow_Per_Source.png"

In [40]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],realSol[2:end-1,2:end-1];levels=32,aspect_ratio = :equal) #(-2:0.1:0.5)
plot!(size=figureSize)
savefig(computationID * "Correct.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/MulP_CylinderLow_Per_Correct.png"

In [41]:
pythonplot()
plot(res)
plot!(yscale=:log10, minorgrid=true)
savefig(computationID * "Residual.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/MulP_CylinderLow_Per_Residual.png"

In [42]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(errors).+1e-12);levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "Error.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/MulP_CylinderLow_Per_Error.png"

In [43]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],pTest.x[2:end-1,2:end-1];levels=(-2:0.1:0.5),aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "Calculated.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/MulP_CylinderLow_Per_Calculated.png"

In [44]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(pTest.x[2:end-1,2:end-1]));levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "LogCalculated.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/MulP_CylinderLow_Per_LogCalculated.png"

In [45]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(lc[2:end-1,2:end-1]).+1e-12);levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "LocalResidul.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/MulP_CylinderLow_Per_LocalResidul.png"

In [46]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],OriginalBDIMSource[2:end-1,2:end-1];levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "ReconstructedSource.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/MulP_CylinderLow_Per_ReconstructedSource.png"

In [47]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(OriginalBDIMSource[2:end-1,2:end-1]).+1e-12);levels=(-12:0.25:-2),aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "LogReconstructedSource.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/MulP_CylinderLow_Per_LogReconstructedSource.png"

In [48]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(originalSource[2:end-1,2:end-1]).+1e-12);levels=(-12:0.25:-2),aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "LogSource.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/MulP_CylinderLow_Per_LogSource.png"

In [49]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(originalSource[2:end-1,2:end-1]-BDIMSource[2:end-1,2:end-1]));levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "DiffSource.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/MulP_CylinderLow_Per_DiffSource.png"

In [50]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(OriginalBDIMSource[2:end-1,2:end-1]./(originalSource[2:end-1,2:end-1].+1e-12)));levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "SourceRatio.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/MulP_CylinderLow_Per_SourceRatio.png"

In [51]:
pythonplot()
plot(pTest.x[:,Int(N[2]/2)])
# plot(reconstructedSource[:,Int(N[2]/2)]+reconstructedSource[:,Int(N[2]/2+1)])
# plot(μ₀[:,Int(N[2]/2),2])
# plot(error[:,Int(N[2]/2)])
plot!(minorgrid=true)
savefig(computationID * "CenterP.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/MulP_CylinderLow_Per_CenterP.png"

In [52]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],mu;levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "Kernel.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/MulP_CylinderLow_Per_Kernel.png"

In [53]:
μ₀[Int(Ng[1]/2-R):Int(Ng[1]/2+R),Int(Ng[2]/2-R):Int(Ng[2]/2+R),2]

129×129 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 ⋮                        ⋮              ⋱                 ⋮              
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   

In [54]:
save(computationID * "sol.jld", "data", pTest.x[2:end-1,2:end-1])

In [55]:
middleSol = load(computationID * "sol_middle.jld")["data"];
downSol = load(computationID * "sol_down.jld")["data"];
downSolShfit = circshift(downSol,[0,N[1]/2]);

ArgumentError: ArgumentError: No file exists at given path: MulP_CylinderLow_Per_sol_middle.jld

In [56]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(middleSol-downSolShfit).+1e-12);levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "DownMiddleDiff.png")

UndefVarError: UndefVarError: `middleSol` not defined